In [1]:
import splitfolders

splitfolders.ratio("concrete_data", output="Dataset", 
                  seed=1337, ratio=(.8, .2), group_prefix=None, move=False)

Copying files: 7247 files [01:59, 60.62 files/s]


# Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

## VGG16

In [16]:
# Import necessary libraries and modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D

In [8]:
# Define constants and data paths
num_classes = 2
image_size = 224
batch_size = 32
train_data_path = 'Dataset/train'
validation_data_path = 'Dataset/val'

In [9]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

In [10]:
# Data Generator for validation (no augmentation)
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [11]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 5797 images belonging to 2 classes.
Found 1450 images belonging to 2 classes.


In [17]:
# Create the VGG16 base model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
vgg16_base.trainable = False

In [18]:
# Build the sequential model
vgg16_model = Sequential([
    vgg16_base,
    GlobalAveragePooling2D(),  
    Dense(num_classes, activation='softmax')
])

In [42]:
vgg16_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,717,768 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2,054 (8.03 KB)

In [19]:
# Compile the model
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
# Fit the model
vgg16_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

Epoch 1/2
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9869 - loss: 0.0543

c:\Users\nensipansuriya\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


181/181 ━━━━━━━━━━━━━━━━━━━━ 924s 5s/step - accuracy: 0.9869 - loss: 0.0543 - val_accuracy: 0.9965 - val_loss: 0.0262
Epoch 2/2
  1/181 ━━━━━━━━━━━━━━━━━━━━ 7:53 3s/step - accuracy: 1.0000 - loss: 0.0144

c:\Users\nensipansuriya\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


181/181 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 1.0000 - loss: 0.0144 - val_accuracy: 0.9965 - val_loss: 0.0257


In [22]:
# Save the trained model to an .h5 file
vgg16_model.save('vgg16_model.h5')

## ResNet50

In [24]:
from tensorflow.keras.applications import ResNet50

In [25]:
# Create the ResNet50 base model
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
resnet50_base.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


In [26]:
# Build the sequential model
resnet50_model = Sequential([
    resnet50_base,
    GlobalAveragePooling2D(),
    Dense(num_classes, activation='softmax')
])

In [43]:
resnet50_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,600,008 (90.03 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 8,198 (32.03 KB)

In [27]:
# Compile the model
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
# Fit the model
resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

Epoch 1/2
181/181 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.9493 - loss: 0.1358 - val_accuracy: 0.9979 - val_loss: 0.0094
Epoch 2/2
181/181 ━━━━━━━━━━━━━━━━━━━━ 73s 393ms/step - accuracy: 1.0000 - loss: 0.0101 - val_accuracy: 0.9979 - val_loss: 0.0101


In [29]:
# Save the trained model to an .h5 file
resnet50_model.save('resnet50_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


In [30]:
# Import necessary libraries and modules
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess

In [31]:
# Load the saved ResNet50 model
resnet_model = load_model('resnet50_model.h5')

In [32]:
# Define test data path
test_data_path = 'Dataset/val'

In [33]:
# Test data generator for ResNet50
resnet_test_datagen = ImageDataGenerator(
    preprocessing_function=resnet_preprocess
)

resnet_test_generator = resnet_test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    shuffle=False,
    class_mode='categorical'
)

Found 1450 images belonging to 2 classes.


In [34]:
# Test data generator for VGG16
vgg_test_datagen = ImageDataGenerator(
    preprocessing_function=vgg_preprocess
)

vgg_test_generator = vgg_test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    shuffle=False,
    class_mode='categorical'
)

Found 1450 images belonging to 2 classes.


In [36]:
# Evaluate the ResNet50 model
resnet_performance = resnet_model.evaluate(resnet_test_generator)
print(f"ResNet50 Model Performance - Loss: {resnet_performance[0]:.2f}, Accuracy: {resnet_performance[1]:.2f}")

# Evaluate the VGG16 model
vgg_performance = vgg16_model.evaluate(vgg_test_generator)
print(f"VGG16 Model Performance - Loss: {vgg_performance[0]:.2f}, Accuracy: {vgg_performance[1]:.2f}")

c:\Users\nensipansuriya\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 47s 971ms/step - accuracy: 0.9946 - loss: 0.0203
ResNet50 Model Performance - Loss: 0.01, Accuracy: 1.00
46/46 ━━━━━━━━━━━━━━━━━━━━ 176s 4s/step - accuracy: 0.9946 - loss: 0.0420
VGG16 Model Performance - Loss: 0.03, Accuracy: 1.00


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

In [37]:
# Import necessary libraries
import numpy as np

In [39]:
# Make predictions on the test set using the VGG16 model
vgg_predictions = vgg16_model.predict(vgg_test_generator)

46/46 ━━━━━━━━━━━━━━━━━━━━ 171s 4s/step


In [40]:
# Get the class predictions for the first five images
first_five_predictions = np.argmax(vgg_predictions[:5], axis=1)
class_labels = ['Negative', 'Positive'] 

In [41]:
print("Predictions for the first five test images:")
for prediction in first_five_predictions:
    print(class_labels[prediction])

Predictions for the first five test images:
Negative
Negative
Negative
Negative
Negative
